In [2]:
%matplotlib inline 
from collections import OrderedDict
import numpy as np
import pandas as pd

def iter_prefixes(t, i): 
    "from longest to shortest"

    for j in reversed(range(i+1, len(t) + 1)):
        yield t[i:j]


def ms(t, s, i): 
    """the longest prefix of t[i:] that occurs in s"""
    for prefix in iter_prefixes(t, i): 
        #print s + ": " + prefix, 
        if prefix in s:
            #print "*"
            return len(prefix)
        #print
    return 0


def bwt(s):
    from collections import Counter
    s = s + "#" 
    A = np.array([list(s[i:] + s[:i]) for i in range(len(s))])
    As = np.vstack(sorted(A, key=tuple))
    CF = Counter(As[:, 0]) 
    S = sorted(CF)
    C = [0] * (len(CF) + 1)

    for i in range(len(S)):
        if i == 0:
            continue
        C[i] = C[i-1] + CF[S[i-1]]
    C[i+1] = len(s)
    return "".join(As[:, len(s) - 1]), "".join(S), "-".join(map(str, C)), As

def table1(t, s):
    a = pd.DataFrame(OrderedDict([('i', [-1] + range(len(t))),
                                  ('t_i', [''] + list(t)), 
                                  ('MS', [1] + map(lambda i: ms(t, s, i), range(len(t))))]))
    a['nzeros'] = [0] + (a[1:].MS.values - a[0:len(t)].MS.values + 1).tolist()
    a['ms'] = [''] + map(lambda i: "".join(['0'] * i) + '1', a.nzeros[1:])
    a['runs'] = [-1] + map(lambda s: int(s == '1'), a.ms[1:])
    return a.set_index('i')

def table2(t, s):
    bwt_s, alp, C, sa = bwt(s)

    a = pd.DataFrame(OrderedDict([('i', range(len(s) + 1)), 
                                  ('s_i', list(s + "#")), 
                                  ('BWT', list(bwt_s)),
                                  ('SA', map(lambda i: len(s) - sa[i].tolist().index("#"), range(sa.shape[0]))),
                                  ('suff_SA', map(lambda i: "".join(sa[i][:sa[i].tolist().index("#")+1].tolist()), range(sa.shape[0])))
                                 ]))
    return a.set_index('i')
    

#t = 'ceededdbe'
#s = 'cbeeeddba'
s = 'dcdceabdcb'
t = 'dcdceabdcbdcdceabdcb'
#table2(t, s[::-1]).head()

/Users/denas/arch/Darwin_x86_64/lib/python/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [3]:
tt = table1(t, s)
#tt['pre_merge_RUNS'] = [-1] + [0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1,1,1,1,0,1,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1,1,1,1,0,1,1,1,0,0,1,1]
#tt['error'] = map(int, tt.pre_merge_RUNS != tt.runs)
tt#[tt.error == 1]

,t_i,MS,nzeros,ms,runs
i,,,,,
-1,,1,0,,-1
0,d,10,10,00000000001,0
1,c,9,0,1,1
2,d,8,0,1,1
3,c,7,0,1,1
4,e,6,0,1,1
5,a,5,0,1,1
6,b,4,0,1,1
7,d,3,0,1,1


In [4]:
table2(t, s)#[50:80]

,s_i,BWT,SA,suff_SA
i,,,,
0,d,b,10,#
1,c,e,5,abdcb#
2,d,c,9,b#
3,c,a,6,bdcb#
4,e,d,8,cb#
5,a,d,1,cdceabdcb#
6,b,d,3,ceabdcb#
7,d,b,7,dcb#
8,c,#,0,dcdceabdcb#


In [5]:
table2(t, s[::-1])

,s_i,BWT,SA,suff_SA
i,,,,
0,b,d,10,#
1,c,b,4,aecdcd#
2,d,d,3,baecdcd#
3,b,#,0,bcdbaecdcd#
4,a,d,8,cd#
5,e,b,1,cdbaecdcd#
6,c,e,6,cdcd#
7,d,c,9,d#
8,c,c,2,dbaecdcd#
